## Serial Monitor for AMS-CB
Version 0.1
# Packages
- pyserial
- numpy


In [ ]:
import serial as pyserial
import keyboard
import numpy as np
from IPython.display import display, HTML

class receiver():
    def __init__(self, port, baudrate, number_of_clients):
        self.ser = pyserial.Serial(port, baudrate)
        self.data = []
        self.channel = 3
        self.clients = number_of_clients
        self.coeff = [6.87565181e-40,-9.27411410e-35,5.48516319e-30,-1.87118391e-25,4.07565006e-21,-5.93033515e-17,5.86752736e-13,-3.95278974e-09,1.80075051e-05,-5.76649020e-02,1.65728946e+02]

        self.totalVoltage = 0
        self.highestCellVoltage = 0
        self.lowestCellVoltage = 0
        self.meanCellVoltage = 0
        self.highestCellTemp = 0
        self.lowestCellTemp = 0
        self.meanCellTemp = 0
        self.actualCurrent = 0
        self.CurrentCounter = 0
        self.status = 0
        self.error = 0
        self.time_per_cycle = 0
        self.adbms_itemp = 0
        self.balance_cells = []
        self.volt_buffer = []
        self.temp_buffer = []
        self.runing = '⚫︎'

    def __str__(self):
        html_text = f'''<b>Battery Monitor</b> {self.blink()}<br>
        total Voltage: {self.totalVoltage:.1f} V <br>
        highest Cell Voltage: {self.highestCellVoltage:.2f} V <br>
        lowest Cell Voltage: {self.lowestCellVoltage:.2f} V <br>
        mean Cell Voltage: {self.meanCellVoltage:.2f} V <br>
        all Voltages: {self.array_to_html_table(18, self.volt_buffer)} <br>
        <br>
        highest Cell Temp: {self.highestCellTemp:.1f} °C <br>
        lowest Cell Temp: {self.lowestCellTemp:.1f} °C <br>
        mean Cell Temp: {self.meanCellTemp:.1f} °C <br>
        all Temps: {self.array_to_html_table(8, self.temp_buffer)} <br>
        <br>
        actual current: {self.actualCurrent} mA <br>
        current counter: {self.CurrentCounter:.2f} Ah <br>
        highest adbms Temp: {self.adbms_itemp} °C <br>
        time per measurement: {self.time_per_cycle} ms <br>
        status code: {self.status:b} <br>
        error code: {self.error:b} <br>'''

        relays_text = 'AIR pos: '
        if(self.status & 8):
            relays_text += '1'
        else:
            relays_text += '0'
        relays_text += '<br> AIR neg: '
        if(self.status & 16):
            relays_text += '1'
        else:
            relays_text += '0'
        relays_text += '<br> Prechar: '
        if(self.status & 32):
            relays_text += '1'
        else:
            relays_text += '0'
        relays_text += '<br>'
        html_text += relays_text

        return html_text

    def __delattr__(self):
        self.ser.close()

    def blink(self):
        if self.runing == ' ':
            self.runing = '⚫︎'
        else:
            self.runing = ' '
        return self.runing

    def calc_temp(self, volt):
        if volt > 23000:
            return 0
        elif volt < 2000:
            return 100
        else:
            return np.polyval(self.coeff, volt)
        
    def array_to_html_table(self, rows, data):
        size = [rows, self.clients]
        required_elements = size[0] * size[1]
        if len(data) < required_elements:
            data += [""] * (required_elements - len(data))
        elif len(data) > required_elements:
            data = data[:required_elements]
        # Generate HTML table
        html = "<table border='1' style='border-collapse: collapse;'>"
        for row in range(size[1]):
            html += "<tr>"
            for col in range(size[0]):
                if rows == 18:
                    if((row==7 and col==16) or (row==7 and col==17)):
                        html += f"<td style='text-align: center;'> nc </td>"
                    else:
                        style = ''
                        if data[row * size[0] + col] == self.highestCellVoltage or data[row * size[0] + col] == self.lowestCellVoltage:
                            style = f" style='background-color: rgb(10, 70, 0);'"
                        if data[row * size[0] + col] > 4.2 or data[row * size[0] + col] < 3.0:
                            style = f" style='background-color: rgb(200, 30, 0);'"
                        if (self.balance_cells[row] & (2**col)):
                            style = f" style='background-color: rgb(40, 0, 230);'"
                        html += f"<td {style}>{data[row * size[0] + col]:.3f} V</td>"
                elif rows == 8:
                    if((row==0 and col==1) or (row==3 and col==2)):
                        html += f"<td style='text-align: center;'> nc </td>"
                    else:
                        style = ''
                        if data[row * size[0] + col] == self.highestCellTemp or data[row * size[0] + col] == self.lowestCellTemp:
                            style = f" style='background-color: rgb(10, 70, 0);'"
                        if data[row * size[0] + col] > 60 or data[row * size[0] + col] < 20:
                            style = f" style='background-color: rgb(200, 0, 0);'"
                        html += f"<td {style}>{data[row * size[0] + col]:.1f} °C</td>"
                else:
                    html += f"<td>{data[row * size[0] + col]}</td>"
            html += "</tr>"
        html += "</table>"
        return html
    
    def read(self):
        cnt = 0
        self.data = []
        while cnt<1000:
            byte = self.ser.read()
            byte = int.from_bytes(byte, byteorder='big', signed=False)
            cnt +=1
            #print(f'byte {byte}')
            if byte == 0xFF:
                byte = self.ser.read()
                byte = int.from_bytes(byte, byteorder='big', signed=False)
                #self.data.append(byte)
                if (byte&0xF0) == (0xA0):
                    stop_command = 0xB0|(byte&0x0F)
                    cnt = 0
                    while cnt<1000:
                        bytel = self.ser.read()
                        bytel = int.from_bytes(bytel, byteorder='big', signed=False)
                        byteh = self.ser.read()
                        byteh = int.from_bytes(byteh, byteorder='big', signed=False)
                        if bytel == 0xFF and byteh == stop_command:
                            return self.data
                        else:
                            #data = (bytel*256 + byteh) *0.0001
                            self.data.append(bytel)
                            self.data.append(byteh)
                else:
                    return self.data
        return 'timeout'
    
    def read2(self):
        try:
            cnt = 0
            self.data = []
            while len(self.data) < (28+self.clients*4+self.clients*18*2+self.clients*8*2):
                self.read()
                cnt +=1
            if cnt>10:
                return 1
            else:
                return 0
        except:
            self.close()
            print('connection lost')
            return 1
        
    def to_signed_32bit(self, value):
        # Interpret the highest bit as the sign bit
        if value & 0x80000000:
            value -= 0x100000000
        return value
    
    def convert_data(self):
        self.totalVoltage = (self.data[0] + self.data[1]*256) *0.1      # in V
        self.highestCellVoltage = (self.data[2] + self.data[3]*256) *0.0001  # in V
        self.lowestCellVoltage = (self.data[4] + self.data[5]*256) *0.0001 # in V
        self.meanCellVoltage =(self.data[6] + self.data[7]*256) *0.0001 # in V
        self.highestCellTemp = self.calc_temp(self.data[8] + self.data[9]*256) # in °C
        self.lowestCellTemp = self.calc_temp(self.data[10] + self.data[11]*256) # in °C
        self.meanCellTemp = self.calc_temp(self.data[12] + self.data[13]*256) # in °C
        self.status = self.data[14]
        self.error = self.data[15]
        self.actualCurrent = (self.data[16] + self.data[17]*(2**8) + self.data[18]*(2**16) + self.data[19]*(2**24)) # in mA
        self.actualCurrent = self.to_signed_32bit(self.actualCurrent)
        self.CurrentCounter = (self.data[20] + self.data[21]*2**8 + self.data[22]*2**16 + self.data[23]*2**24) # in As
        self.CurrentCounter = self.to_signed_32bit(self.CurrentCounter)/3600
        self.time_per_cycle = (self.data[24] + self.data[25]*256)  # in ms
        self.adbms_itemp = (self.data[26] + self.data[27]*256)  # in °C
        offset = 28

        self.balance_cells = []
        for i in range(self.clients):
            self.balance_cells.append((self.data[offset +i*4] +self.data[offset+1 +i*4]*(2**8) +self.data[offset+2 +i*4]*(2**16) + self.data[offset+3 +i*4]*(2**24)) )
        offset += self.clients*4

        self.volt_buffer = []
        for i in range(self.clients*18):
            self.volt_buffer.append((self.data[offset +i*2] +self.data[offset+1 +i*2]*256) *0.0001)
        offset += self.clients*18*2
        self.temp_buffer = []
        for i in range(self.clients*8):
            self.temp_buffer.append(self.calc_temp(self.data[offset +i*2] + self.data[offset+1 +i*2]*256))
        offset += self.clients*8*2
    
    def close(self):
        self.ser.close()

stm32 = receiver('/dev/ttyACM0', 115200, number_of_clients=2)   # adjust com port and used segments here
display1 = display('starting...', display_id=True)

try:
    while True:
        if stm32.read2():
            break
        stm32.convert_data()
        display1.update(HTML(str(stm32)))
except KeyboardInterrupt:
    stm32.close()


connection lost
